In [ ]:
"""
Similar to `store-pretrained-model-paths.ipynb`, but also exports hidden states. Use for clustering analysis to compare clusters of hidden states versus expert IDs.
"""
None

In [ ]:
"""
Imports
"""
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.loss.loss_utils import ForCausalLMLoss # Cross-entropy loss that handles label shifting
from datasets import load_dataset, interleave_datasets
import pandas as pd
import numpy as np
from tqdm import tqdm
from termcolor import colored
import importlib 

from utils.memory import check_memory, clear_all_cuda_memory
from utils.store_topk import convert_topk_to_df
from utils.store_outputs import convert_outputs_to_df
from utils import pretrained_models

# https://docs.rapids.ai/install/
import cupy
import cudf
import cuml

main_device = 'cuda:0'
seed = 1234
clear_all_cuda_memory()
check_memory()

## Load base model

In [ ]:
"""
Load the base tokenizer/model

Architectures supported currently:
- OlMoE architecture, includes OLMoE-1B-7B-0125-Instruct (1B/7B)
- Qwen2MoE architecture, inclues Qwen1.5-MoE-A2.7B-Chat (2.7B/14.3B), Qwen2-57B-A14B (14B/57B)
- Deepseek v2 architecture, includes Deepseek-v2-Lite (2.4B/15.7B), Deepseek-v2 (21B/236B)
- Deepseek v3 architecture, includes Deepseek-v3 (37B/671B), Deepseek-R1 (37B/671B), Moonlight-16B-A3B (3B/16B)
"""
selected_model_index = 0

def get_model(index):
    model = [
        ('allenai/OLMoE-1B-7B-0125-Instruct', 'olmoe', 'olmoe'),
        ('Qwen/Qwen1.5-MoE-A2.7B-Chat', 'qwen1.5moe', 'qwen2moe'),
        ('deepseek-ai/DeepSeek-V2-Lite', 'dsv2', 'dsv2'),
        ('moonshotai/Moonlight-16B-A3B', 'moonlight', 'dsv3')
    ][index]

    return model[0], model[1], model[2]

model_id, model_prefix, model_architecture = get_model(selected_model_index)
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token = False, add_bos_token = False, padding_side = 'left', trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16, trust_remote_code = True).cuda().eval()

In [ ]:
"""
Load reverse-engineered forward pass functions that return topk expert IDs and weights
"""
model_module = importlib.import_module(f"utils.pretrained_models.{model_architecture}")
run_model_return_topk = getattr(model_module, f"run_{model_architecture}_return_topk")

def test_custom_forward_pass(model, pad_token_id):
    inputs = tokenizer(['Hi! I am a dog and I like to bark', 'Vegetables are good for'], return_tensors = 'pt', padding = 'max_length', truncation = True, max_length = 512).to(model.device)
    original_results = model(**inputs)
    custom_results = run_model_return_topk(model, inputs['input_ids'], inputs['attention_mask'], return_hidden_states = True)
    assert torch.equal(original_results.logits, custom_results['logits']), 'Error in custom forward'
    assert len(custom_results['all_topk_experts']) == len(custom_results['all_topk_weights']), 'Length of topk IDs and weights not equal'
    print(f"Length of topk: {len(custom_results['all_topk_experts'])}")
    print(f"Topk size: {custom_results['all_topk_experts'][0].shape}")
    print(f"First token topk IDs: {custom_results['all_topk_experts'][0][1,]}")
    print(f"First token topk weights: {custom_results['all_topk_weights'][0][1,]}")
    loss = ForCausalLMLoss(custom_results['logits'], torch.where(inputs['input_ids'] == pad_token_id, torch.tensor(-100), inputs['input_ids']), model.config.vocab_size).detach().cpu().item()
    print(f"LM loss: {loss}")
    print(f"Hidden states layers (pre-mlp, post-layer): {len(custom_results['all_pre_mlp_hidden_states'])} | {len(custom_results['all_hidden_states'])}")
    print(f"Hidden state size (pre-mlp, post-layer): {(custom_results['all_pre_mlp_hidden_states'][0].shape)} | {(custom_results['all_hidden_states'][0].shape)}")

test_custom_forward_pass(model, tokenizer.pad_token_id)

## Get dataset

In [ ]:
"""
Load dataset
"""
def load_raw_ds():
   
    ds_en = load_dataset('allenai/c4', 'en', split = 'validation', streaming = True).shuffle(seed = 123, buffer_size = 1_000_000)
    ds_zh = load_dataset('allenai/c4', 'zh', split = 'validation', streaming = True).shuffle(seed = 123, buffer_size = 1_000_000)
    ds_es = load_dataset('allenai/c4', 'es', split = 'validation', streaming = True).shuffle(seed = 123, buffer_size = 1_000_000)
    
    def get_data(ds, n_samples):
        raw_data = []
        ds_iter = iter(ds)
        for _ in range(0, n_samples):
            sample = next(ds_iter, None)
            if sample is None:
                break
            raw_data.append(sample['text'])
        
        return raw_data
    
    return get_data(ds_en, 500) + get_data(ds_zh, 100) + get_data(ds_es, 100)


raw_data = load_raw_ds()

In [ ]:
""" 
Load dataset into a dataloader. The dataloader returns the original tokens - this is important for BPE tokenizers as otherwise it's difficult to reconstruct the correct string later!
"""
from torch.utils.data import Dataset, DataLoader

class ReconstructableTextDataset(Dataset):

    def __init__(self, text_dataset, tokenizer, max_length):
        """
        Creates a dataset object that also returns a B x N list of the original tokens in the same position as the input ids.

        Params:
            @text_dataset: A list of B samples of text dataset.
            @tokenizer: A HF tokenizer object.
        """
        tokenized = tokenizer(text_dataset, add_special_tokens = False, max_length = max_length, padding = 'max_length', truncation = True, return_offsets_mapping = True, return_tensors = 'pt')

        self.input_ids = tokenized['input_ids']
        self.attention_mask = tokenized['attention_mask']
        self.offset_mapping = tokenized['offset_mapping']
        self.original_tokens = self.get_original_tokens(text_dataset)

    def get_original_tokens(self, text_dataset):
        """
        Return the original tokens associated with each B x N position. This is important for reconstructing the original text when BPE tokenizers are used.
        
        Params:
            @input_ids: A B x N tensor of input ids.
            @offset_mapping: A B x N x 2 tensor of offset mappings. Get from `tokenizer(..., return_offsets_mapping = True)`.

        Returns:
            A list of length B, each with length N, containing the corresponding original tokens corresponding to the token ID at the same position of input_ids.
        """
        all_token_substrings = []
        for i in range(0, self.input_ids.shape[0]):
            token_substrings = []
            for j in range(self.input_ids.shape[1]): 
                start_char, end_char = self.offset_mapping[i][j].tolist()
                if start_char == 0 and end_char == 0: # When pads, offset_mapping might be [0, 0], so let's store an empty string for those positions.
                    token_substrings.append("")
                else:
                    original_substring = text_dataset[i][start_char:end_char]
                    token_substrings.append(original_substring)
            
            all_token_substrings.append(token_substrings)

        return all_token_substrings

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {'input_ids': self.input_ids[idx], 'attention_mask': self.attention_mask[idx], 'original_tokens': self.original_tokens[idx]}
    
def collate_fn(batch):
    """
    Custom collate function; necessary to return original_tokens in the correct shape 
    """
    input_ids = torch.stack([b['input_ids'] for b in batch], dim = 0)
    attention_mask = torch.stack([b['attention_mask'] for b in batch], dim = 0)        
    original_tokens = [b['original_tokens'] for b in batch]
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'original_tokens': original_tokens}

test_dl = DataLoader(
    ReconstructableTextDataset(raw_data, tokenizer, max_length = 1024),
    batch_size = 8,
    shuffle = False,
    collate_fn = collate_fn
)

## Get expert selections + export

In [ ]:
""" 
Run forward passes + export data

Note the bulk of compute time will be spent on exporting the CSV, not handling the forward passes.
"""

@torch.no_grad()
def run_and_export_topk(model, model_prefix, file_suffix, dl, layers_to_keep, max_batches = None):
    """
    Run forward passes on a given model ID, return topk df
    """
    b_count = 0
    all_pre_mlp_hidden_states = []
    sample_dfs = []
    topk_dfs = []

    for batch_ix, batch in tqdm(enumerate(dl), total = len(dl)):

        input_ids = batch['input_ids'].to(main_device)
        attention_mask = batch['attention_mask'].to(main_device)
        original_tokens = batch['original_tokens']

        output = run_model_return_topk(model, input_ids, attention_mask, return_hidden_states = True)

        # Check no bugs by validating output/perplexity
        if batch_ix == 0:
            loss = ForCausalLMLoss(output['logits'], torch.where(input_ids == tokenizer.pad_token_id, torch.tensor(-100), input_ids), model.config.vocab_size).detach().cpu().item()
            for i in range(min(2, input_ids.size(0))):
                decoded_input = tokenizer.decode(input_ids[i, :attention_mask[i].sum()], skip_special_tokens = True)
                next_token_id = torch.argmax(output['logits'][i, -1, :]).item()
                print(decoded_input + colored(tokenizer.decode([next_token_id], skip_special_tokens = True), 'green'))
            print(f"PPL:", torch.exp(torch.tensor(loss)).item())
        
        original_tokens_df = pd.DataFrame(
            [(seq_i, tok_i, tok) for seq_i, tokens in enumerate(original_tokens) for tok_i, tok in enumerate(tokens)], 
            columns = ['sequence_ix', 'token_ix', 'token']
        )

        # Create sample (token) level dataframe
        sample_df =\
            convert_outputs_to_df(input_ids, attention_mask, output['logits'])\
            .merge(original_tokens_df, how = 'left', on = ['token_ix', 'sequence_ix'])\
            .assign(batch_ix = batch_ix)

        # # We'll analyze the first 5 layers only for simplicity
        # pre_mlp_hidden_state_df =\
        #     convert_hidden_states_to_df(input_ids, attention_mask, output['all_pre_mlp_hidden_states'][0:5])\
        #     .assign(batch_ix = batch_ix)\
        #     .drop(columns = 'token_id')

        topk_df =\
            convert_topk_to_df(input_ids, attention_mask, output['all_topk_experts'], output['all_topk_weights'])\
            .assign(batch_ix = batch_ix, weight = lambda df: df['weight'])\
            .drop(columns = 'token_id')\
            .pipe(lambda df: df[df['layer_ix'].isin(layers_to_keep)])
        
        sample_dfs.append(sample_df)
        all_pre_mlp_hidden_states.append([output['all_pre_mlp_hidden_states'][i] for i in layers_to_keep])
        topk_dfs.append(topk_df)
        # sample_df.to_csv(f'data/hidden-states/{model_prefix}-{file_suffix}-samples.csv', mode = 'w' if batch_ix == 0 else 'a', index = False, header = (batch_ix == 0))
        # all_pre_mlp_hidden_states.append(output['all_pre_mlp_hidden_states'][0:5])
        # pre_mlp_hidden_state_df.to_csv(f'data/hidden-states/{model_prefix}-{file_suffix}-pre-mlp-hidden-states.csv', mode = 'w' if batch_ix == 0 else 'a', index = False, header = (batch_ix == 0))
        # topk_df.to_csv(f'data/hidden-states/{model_prefix}-{file_suffix}-routes.csv', mode = 'w' if batch_ix == 0 else 'a', index = False, header = (batch_ix == 0))
        # topk_df[topk_df['topk_ix'] == 1].to_csv(f'data/hidden-states/{model_prefix}-{file_suffix}-routes-top1.csv', mode = 'w' if batch_ix == 0 else 'a', index = False, header = (batch_ix == 0))

        b_count += 1
        if max_batches is not None and b_count >= max_batches:
            break

    return {'sample_df': pd.concat(sample_dfs), 'topk_df': pd.concat(topk_dfs), 'all_pre_mlp_hidden_states': all_pre_mlp_hidden_states}

res = run_and_export_topk(model, model_prefix, 'fw', test_dl, layers_to_keep = list(range(0, 5)), max_batches = None)

## Prep for Clustering

In [ ]:
"""
Let's clean up the mappings here. We'll get everything to a sample_ix level first.
"""

sample_df_raw =\
    res['sample_df']\
    .assign(sample_ix = lambda df: df.groupby(['batch_ix', 'sequence_ix', 'token_ix']).ngroup())\
    .assign(seq_id = lambda df: df.groupby(['batch_ix', 'sequence_ix']).ngroup())\
    .reset_index()

topk_df =\
    res['topk_df']\
    .merge(sample_df_raw[['sample_ix', 'batch_ix', 'sequence_ix', 'token_ix']], how = 'inner', on = ['sequence_ix', 'token_ix', 'batch_ix'])\
    .drop(columns = ['sequence_ix', 'token_ix', 'batch_ix'])

sample_df =\
    sample_df_raw\
    .drop(columns = ['batch_ix', 'sequence_ix'])

display(topk_df)
display(sample_df)

In [ ]:
"""
We need something to map our hidden state back onto the sample_df_raw batch_ix, sequence_ix, token_ix so that we can skip masked tokens

Let's get all unique sequence_ix, batch_ix in sample_df_raw (assumes all sequences have at least one non-masked token)
Then, we'll join onto all possible token_ix (assumes all token_ix have been represented at least one in sample_df_raw)
"""
merge_indices =\
    sample_df_raw[['batch_ix', 'sequence_ix']]\
    .drop_duplicates()\
    .merge(pd.DataFrame({'token_ix': sorted(sample_df_raw['token_ix'].drop_duplicates().tolist())}), how = 'cross')\
    .sort_values(by = ['batch_ix', 'sequence_ix', 'token_ix'])\
    .assign(original_index = lambda df: df.index)\
    .merge(
        sample_df_raw[['sample_ix', 'batch_ix', 'sequence_ix', 'token_ix']].assign(is_valid = 1),
        how = 'left',
        on = ['batch_ix', 'sequence_ix', 'token_ix']
    )\
    .pipe(lambda df: df[df['is_valid'] == 1])\
    .reset_index()

merge_indices

In [ ]:
"""
Now we can get the hidden states by layer, removing all the attention masked indices
"""
hs_by_layer = {}
for layer_ix in range(0, len(res['all_pre_mlp_hidden_states'][0])):
    hs_by_layer[layer_ix] = torch.cat(
        [batch_res[layer_ix] for batch_res in res['all_pre_mlp_hidden_states']], # A list of length num_batches, each of size BN x D
        dim = 0
    ) # Creates a list of n_all_tokens x D
    hs_by_layer[layer_ix] = hs_by_layer[layer_ix][merge_indices['original_index'].tolist(), :]

hs_by_layer[0].shape

## Clustering

In [ ]:
""" 
Topk = 1 clusters
"""
topk_wide =\
    topk_df\
    .pipe(lambda df: df[df['topk_ix'] == 1])\
    .pipe(lambda df: df[df['layer_ix'] <= 4])\
    .merge(sample_df[['sample_ix', 'token']], on = 'sample_ix', how = 'inner')\
    .pivot(index = ['sample_ix', 'token'], columns = 'layer_ix', values = 'expert')\
    .rename(columns = lambda c: f'layer_{c}_id')\
    .reset_index()

display(topk_wide.groupby('layer_1_id', as_index = False).agg(n_samples = ('token', 'size')).sort_values(by = 'n_samples', ascending = False))

topk_wide\
    .groupby(['layer_0_id', 'layer_1_id', 'layer_2_id', 'layer_3_id', 'layer_4_id'], as_index = False)\
    .agg(
        n_samples = ('token', 'size'),
        samples = ('token', lambda s: s.sample(n = min(len(s), 10)).tolist())
    )\
    .pipe(lambda df: df[df['n_samples'] >= 3])

In [ ]:
hs_cupy = cupy.asarray(hs_by_layer[0].to(torch.float32))

kmeans_model = cuml.cluster.KMeans(
    n_clusters = 64,
    random_state = 123,
    verbose = True
)

kmeans_model.fit(hs_cupy)

labels_cupy = kmeans_model.labels_

labels_cupy.tolist()

In [ ]:
"""
K-Means (note - returns imbalanced clusters)
""" 
from sklearn.cluster import MiniBatchKMeans

def cluster_kmeans(hs_for_layer: torch.Tensor, n_clusters = 64):
    """
    Params:
        @hs_for_layer: A n_token_samples x D tensor
        @n_clusters: The number of clusters to return

    Returns:
        A list of length n_token_samples of cluster ids
    """
    hs_cupy = cupy.asarray(hs_for_layer.to(torch.float32))
    kmeans_model = cuml.cluster.KMeans(
        n_clusters = n_clusters,
        max_iter = 1000,
        random_state = 123,
        verbose = True
    )
    kmeans_model.fit(hs_cupy)
    cluster_labels = kmeans_model.labels_ # shape = (n_samples,)
    # cluster_centers = kmeans_model.cluster_centers_ # shape = (num_clusters, D)
    return cluster_labels.tolist()

kmeans_res = [{'cluster_method': 'kmeans', 'layer_ix': k, 'cluster_ids': cluster_kmeans(v, 64)} for k, v in tqdm(hs_by_layer.items())]

joined_df =\
    pd.concat([pd.DataFrame({'layer_' + str(x['layer_ix']) + '_id': x['cluster_ids']}) for x in kmeans_res], axis = 1)\
    .assign(sample_ix = merge_indices['sample_ix'].tolist())\
    .merge(sample_df, on = 'sample_ix', how = 'inner')

display(joined_df.groupby('layer_1_id', as_index = False).agg(n_samples = ('token', 'size')).sort_values(by = 'n_samples', ascending = False))
display(joined_df.groupby('layer_3_id', as_index = False).agg(n_samples = ('token', 'size')).sort_values(by = 'n_samples', ascending = False))

routes =\
    joined_df\
    .groupby(['layer_1_id', 'layer_2_id', 'layer_3_id', 'layer_4_id'], as_index = False)\
    .agg(n_samples = ('token', 'size'), samples = ('token', lambda s: s.sample(n = min(len(s), 10)).tolist()))\
    .pipe(lambda df: df[df['n_samples'] >= 3])
    
display(routes)
routes.to_csv('test-kmeans.csv')

In [ ]:
# """
# Balanced k-means
# """
# from k_means_constrained import KMeansConstrained

# def cluster_kmeans_balanced(hs_for_layer: torch.Tensor, n_clusters = 64):
#     """
#     Params:
#         @hs_for_layer: A n_token_samples x D tensor
#         @n_clusters: The number of clusters to return

#     Returns:
#         A list of length n_token_samples of cluster ids
#     """
#     hs_layer_np = hs_for_layer.to(torch.float32).numpy()
#     km = KMeansConstrained(
#         n_clusters = n_clusters,
#         size_min = len(hs_layer_np) // (n_clusters * 5),  # 20% of the uniform distribution value
#         random_state = 123
#     )
#     km.fit(hs_layer_np)
#     cluster_labels = km.labels_ # shape = (n_samples,)
#     return cluster_labels.tolist()

# kmeans_res = [{'cluster_method': 'kmeans', 'layer_ix': k, 'cluster_ids': cluster_kmeans_balanced(v)} for k, v in tqdm(hs_by_layer.items())]

In [ ]:
"""
HDBScan
"""

hs_cupy = cupy.asarray(hs_by_layer[0].to(torch.float32))
# https://docs.rapids.ai/api/cuml/stable/api/#hdbscan
hdbscan_model = cuml.cluster.HDBSCAN(
    min_cluster_size = len(hs_cupy) // (64 * 500), # Min 1/20 of the uniform dist value
    max_cluster_size = len(hs_cupy) // (64 * 1/500), # Max 20x the uniform dist values 
    max_iter = 1000,
    metric = 'euclidean',
    min_samples = 2,
    verbose = True
)
hdbscan_model.fit(hs_cupy)
cluster_labels = hdbscan_model.labels_ # shape = (n_samples,)
# cluster_centers = kmeans_model.cluster_centers_ # shape = (num_clusters, D)

print(len(cluster_labels))
print(len([l for l in cluster_labels.tolist() if l == -1]))

In [ ]:
# """
# HDBSCAN
# """
# import hdbscan

# hs_for_layer = hs_by_layer[0]

# hs_layer_np = hs_for_layer.to(torch.float32).numpy()
# clusterer = hdbscan.HDBSCAN(
#     min_cluster_size = 500,
#     min_samples = 100,
#     metric = 'euclidean' # cosine
#     ) # https://stackoverflow.com/questions/67898039/hdbscan-difference-between-parameters
# labels = clusterer.fit_predict(hs_layer_np)

In [ ]:
"""
UMAP Testiong
"""
import umap
import plotly.express as px

hs_for_layer = hs_by_layer[0]
hs_layer_np = hs_for_layer.to(torch.float32).numpy()[0:50000,]

subset_indicies = np.random.choice(hs_layer_np.shape[0], size = 50_000, replace = False) # 50k = 3min
reducer = umap.UMAP(
    n_components = 2, 
    n_neighbors = 15, 
    min_dist = 0.1, 
    n_epochs = 100,
    random_state = None # Allow parallelism
    )

embed = reducer.fit(hs_layer_np[subset_indicies])  # shape = (n_samples, n_components)


px.scatter(
    pd.concat(
        [pd.DataFrame({'d1': embed[:, 0], 'd2': embed[:, 1]}), sample_df.head(embed.shape[0])],
        axis = 1
        )\
        .sample(5000)
        .assign(
            is_of = lambda df: np.where(df['token'] == ' of', 1, 0)
        ),
    x = 'd1',
    y = 'd2',
    color = 'is_of',
    hover_data = ['token']
)

In [ ]:
"""
UMAP + HDBSCAN
"""
import umap
import hdbscan

hs_for_layer = hs_by_layer[0]

hs_layer_np = hs_for_layer.to(torch.float32).numpy()
subset_indicies = np.random.choice(list(range(0, hs_layer_np.shape[0])), size = 50_000, replace = False) # 50k = 3min
reducer = umap.UMAP(
    n_components = 10, 
    n_neighbors = 15, 
    min_dist = 0.1, 
    n_epochs = 100,
    random_state = None # Allow parallelism
    )

embed = reducer.fit(hs_layer_np[subset_indicies])  # shape = (n_samples, n_components)
embed_all = reducer.transform(hs_layer_np)


In [ ]:
from cuml.manifold.umap import UMAP

hs_for_layer = hs_by_layer[0]

hs_layer_np = hs_for_layer.to(torch.float32).numpy()
subset_indicies = np.random.choice(list(range(0, hs_layer_np.shape[0])), size = 50_000, replace = False) # 50k = 3min
reducer = umap.UMAP(
    n_components = 10, 
    n_neighbors = 15, 
    min_dist = 0.1, 
    n_epochs = 100,
    data_on_host = True
    )

embed = reducer.fit(hs_layer_np[subset_indicies])  # shape = (n_samples, n_components)
embed_all = reducer.transform(hs_layer_np)


In [ ]:
clusterer = hdbscan.HDBSCAN(
    min_cluster_size = 500,
    min_samples = 100,
    metric = 'euclidean' # cosine
    ) # https://stackoverflow.com/questions/67898039/hdbscan-difference-between-parameters
labels = clusterer.fit_predict(embed_all)


In [ ]:
import plotly.express as px

px.scatter(
    pd.concat(
        [
            pd.DataFrame({'d1': embedding[:, 0], 'd2': embedding[:, 1]}),
            sample_df.head(embedding.shape[0])
        ],
        axis = 1
    ).sample(5000)
    .assign(
        is_of = lambda df: np.where(df['token'] == ' of', 1, 0)
    ),
    x = 'd1',
    y = 'd2',
    color = 'is_of',
    hover_data = ['token']
)

In [ ]:
pd.concat(
    [
        pd.DataFrame({'d1': embedding[:, 0], 'd2': embedding[:, 1]}),
        sample_df.head(embedding.shape[0])
    ],
    axis = 1
    )

## Compare to Dense Models